In [4]:
%tensorflow_version 2.x
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization, Activation, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 10
l = 16
num_filter = 24
compression = 0.5
dropout_rate = 0

In [6]:
# Load CIFAR10 Data
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()
img_height, img_width, channel = X_train.shape[1],X_train.shape[2],X_train.shape[3]
# convert to one hot encoing 
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes) 

170500096/170498071 [==============================] - 4s 0us/step


In [0]:
import numpy as np
X_train_mean = np.mean(X_train, axis=(0,1,2))
X_train_std = np.std(X_train, axis=(0,1,2))
X_train = (X_train - X_train_mean) / X_train_std
X_test = (X_test - X_train_mean) / X_train_std

In [0]:
# Dense Block
def denseblock(input, num_filter =32):
    global compression
    temp = input
    for _ in range(l): 
        BatchNorm = layers.BatchNormalization()(temp)
        relu = layers.Activation('relu')(BatchNorm)
        Conv2D_3_3 = layers.Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
            Conv2D_3_3 = layers.Dropout(dropout_rate)(Conv2D_3_3)
        concat = layers.Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

## transition Blosck
def transition(input, num_filter =32):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = layers.Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
         Conv2D_BottleNeck = layers.Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = layers.AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    return avg

#output layer
def output_layer(input):
    global compression
    BatchNorm = layers.BatchNormalization()(input)
    relu = layers.Activation('relu')(BatchNorm)
    AvgPooling = layers.AveragePooling2D(pool_size=(2,2))(relu)
    conv=layers.Conv2D(int(10),(2,2), use_bias=False)(AvgPooling)
    conv1=layers.Conv2D(int(10),(1,1), use_bias=False)(conv)
    flat = layers.Flatten()(conv1)
    output = layers.Activation("softmax")(flat)
    return output

In [0]:
num_filter = 24
l = 16
input = layers.Input(shape=(img_height, img_width, channel,))
First_Conv2D = layers.Conv2D(num_filter, (4,4), use_bias=False ,padding='same',activation=tf.nn.relu,kernel_initializer=tf.keras.initializers.glorot_normal(seed=30))(input)

First_Block = denseblock(First_Conv2D, num_filter)
First_Transition = transition(First_Block, num_filter )

Second_Block = denseblock(First_Transition, num_filter )
Second_Transition = transition(Second_Block, num_filter )

Third_Block = denseblock(Second_Transition, num_filter )
Third_Transition = transition(Third_Block, num_filter )

Last_Block = denseblock(Third_Transition,  num_filter )
output = output_layer(Last_Block)

In [12]:
model = Model(inputs=[input], outputs=[output])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 24)   1152        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 24)   96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 24)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [0]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=15,horizontal_flip=True,width_shift_range=0.1,height_shift_range=0.1,zoom_range=0.3,rescale=1./255.)
train_generator=datagen.flow(X_train,y_train,batch_size=32)
test_generator=datagen.flow(X_test,y_test,batch_size=32)

In [13]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(0.001),
              metrics=['accuracy'])
import os
checkpoint_path="training_1/cp-{epoch:04d}.ckpt" 
checkpoint_dir=os.path.dirname(checkpoint_path)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1,save_weights_only=True)
model.fit(
        train_generator,
        steps_per_epoch=(len(X_train) // 32),
        epochs=200,
        validation_data=test_generator,callbacks=[checkpoint],
        validation_steps=(len(X_test) // 32))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1562 steps, validate for 312 steps
Epoch 1/200
1561/1562 [============================>.] - ETA: 0s - loss: 1.6743 - accuracy: 0.3859
Epoch 00001: val_accuracy improved from -inf to 0.43590, saving model to training_1/cp-0001.ckpt
1562/1562 [==============================] - 112s 72ms/step - loss: 1.6741 - accuracy: 0.3860 - val_loss: 1.5516 - val_accuracy: 0.4359
Epoch 2/200
1561/1562 [============================>.] - ETA: 0s - loss: 1.2963 - accuracy: 0.5330
Epoch 00002: val_accuracy improved from 0.43590 to 0.57993, saving model to training_1/cp-0002.ckpt
1562/1562 [==============================] - 97s 62ms/step - loss: 1.2962 - accuracy: 0.5330 - val_loss: 1.1947 - val_accuracy: 0.5799
Epoch 3/200
1561/1562 [============================>.] - ETA: 0s - loss: 1.0850 - accuracy: 0.6121
Epoch 00003: val_accuracy improved from 0.57993 to 0.64403, saving model to training_1/cp-0003.ckpt
1562/1562 [=============================

In [17]:
checkpoint_path="training_2/cp-{epoch:04d}.ckpt" 
checkpoint_dir=os.path.dirname(checkpoint_path)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1,save_weights_only=True)
model.fit(
        train_generator,
        steps_per_epoch=(len(X_train) // 32),
        epochs=100,
        validation_data=test_generator,callbacks=[checkpoint],
        validation_steps=(len(X_test) // 32))

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 1562 steps, validate for 312 steps
Epoch 1/100
1561/1562 [============================>.] - ETA: 0s - loss: 0.0982 - accuracy: 0.9657
Epoch 00001: val_accuracy improved from -inf to 0.89473, saving model to training_2/cp-0001.ckpt
1562/1562 [==============================] - 90s 58ms/step - loss: 0.0981 - accuracy: 0.9657 - val_loss: 0.4063 - val_accuracy: 0.8947
Epoch 2/100
1561/1562 [============================>.] - ETA: 0s - loss: 0.0998 - accuracy: 0.9654
Epoch 00002: val_accuracy did not improve from 0.89473
1562/1562 [==============================] - 90s 58ms/step - loss: 0.0997 - accuracy: 0.9654 - val_loss: 0.4406 - val_accuracy: 0.8847
Epoch 3/100
1561/1562 [============================>.] - ETA: 0s - loss: 0.1025 - accuracy: 0.9641
Epoch 00003: val_accuracy did not improve from 0.89473
1562/1562 [==============================] - 90s 57ms/step - loss: 0.1025 - accuracy: 0.9641 - val_loss: 0.4094 - val_accuracy: 0.8

In [20]:
loss, acc = model.evaluate(test_generator)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

  ...
    to  
  ['...']
313/313 [==============================] - 7s 22ms/step - loss: 0.3891 - accuracy: 0.9003
Restored model, accuracy: 90.03%
